In [1]:
import calendar
import pandas as pd
import panel as pn
from datetime import date, timedelta
from sqlalchemy import create_engine
from itables import show

pn.extension('tabulator')

engine = create_engine("sqlite:///c:\\ruby\\port_lite\\db\\development.sqlite3")
conlite = engine.connect()

data_path = "../data/"
csv_path = "\\Users\\User\\iCloudDrive\\"
box_path = "\\Users\\User\\Dropbox\\"

cash = 720_000
cash

720000

In [2]:
format_dict = {
    'qty':'{:,}',
    'price':'{:.2f}','target':'{:.2f}','current':'{:.2f}','change':'{:.2f}','diff':'{:.2f}',
    'amount':'{:,.2f}','sell_amt':'{:,.2f}'
}

In [3]:
pd.read_sql_query('SELECT * FROM orders ORDER BY id DESC LIMIT 1', conlite).style.format(format_dict)

,id,trade,name,qty,price,active,reason,market
0,23,S,TOP,"3,000",52.50,0,5%,SET50


In [4]:
cols = 'trade name qty price active reason market'.split()
colt = 'trans name qty target active spd current change percent'.split()

### Print to verify before upload file

In [5]:
sql = '''
SELECT trade, name, qty, price, qty * price AS amount, reason, market, active
FROM orders
ORDER BY trade, name'''
orders = pd.read_sql(sql, conlite)
df_tab = pn.widgets.Tabulator(orders, layout='fit_data', width=800)
df_tab

Tabulator(layout='fit_data', value=   trade    name  ..., width=800)

In [6]:
file_name = 'orders.csv'
data_file = data_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name

orders[cols].to_csv(output_file, header=True, index=False)
orders[cols].to_csv(data_file, header=True, index=False)
orders[cols].to_csv(box_file, header=True, index=False)

### End of transactional process

In [7]:
sql = '''
SELECT trade, name, qty, price, qty * price AS amount, reason, market, active
FROM orders
WHERE active = 2
ORDER BY trade, name'''
df_act_2 = pd.read_sql(sql, conlite)
df_act_2.style.format(format_dict)

,trade,name,qty,price,amount,reason,market,active
0,B,BGC,"10,000",10.40,"104,000.00",DOS,SET,2
1,B,DOHOME,"4,000",22.90,"91,600.00",RD05%,SET100,2
2,B,IMH,"6,000",12.00,"72,000.00",RU09%,mai,2
3,B,ROJNA,"18,000",6.50,"117,000.00",6L,SET,2
4,B,WHART,"10,000",12.50,"125,000.00",DOS,SET,2
5,S,DCC,"40,000",3.00,"120,000.00",3%,SET,2
6,S,IVL,"3,000",44.50,"133,500.00",5%,SET50,2


In [8]:
orders_grp = df_act_2.groupby(['trade'])
orders_grp['amount'].sum()

trade
B    509600.0
S    253500.0
Name: amount, dtype: float64

In [9]:
mask = df_act_2.trade == 'B'
buy_stocks = df_act_2[mask]
buy_stocks.style.format(format_dict)

,trade,name,qty,price,amount,reason,market,active
0,B,BGC,"10,000",10.40,"104,000.00",DOS,SET,2
1,B,DOHOME,"4,000",22.90,"91,600.00",RD05%,SET100,2
2,B,IMH,"6,000",12.00,"72,000.00",RU09%,mai,2
3,B,ROJNA,"18,000",6.50,"117,000.00",6L,SET,2
4,B,WHART,"10,000",12.50,"125,000.00",DOS,SET,2


In [10]:
buy_amt = buy_stocks.amount.sum()
balance = cash - buy_amt 
cash, buy_amt, balance

(720000, 509600.0, 210400.0)

In [11]:
mask = df_act_2.trade == 'S'
sell_stocks = df_act_2[mask]
sell_stocks.style.format(format_dict)

,trade,name,qty,price,amount,reason,market,active
5,S,DCC,"40,000",3.00,"120,000.00",3%,SET,2
6,S,IVL,"3,000",44.50,"133,500.00",5%,SET50,2


In [12]:
sell_amt = round(sell_stocks.amount.sum(),2)
balance = cash + sell_amt 
(cash, sell_amt, balance)

(720000, 253500.0, 973500.0)

### After call ord-log

In [13]:
def categorise(row):  
    if row['trade'] == 'B':
        return 'Buy'
    else:
        return 'Sell'

In [14]:
df = pd.read_csv('../data/orders-log.csv')
df['trans'] = df.apply(lambda row: categorise(row), axis=1)
df['amount'] = df.qty * df.target

In [15]:
show(df[['trans','name','spd','reason','qty','target','amount','current','active']])

<IPython.core.display.Javascript object>

trans,name,spd,reason,qty,target,amount,current,active


In [16]:
mb = (df.trans == 'Buy') & (df.spd >= -3)
df[mb].sort_values(['name'],ascending=[True]).style.format(format_dict)

,trade,name,spd,reason,market,qty,target,current,change,percent,active,trans,amount
0,B,BGC,-3,DOS,SET,"10,000",10.40,10.70,0.10,+0.94%,2,Buy,"104,000.00"
1,B,DOHOME,-3,RD05%,SET100,"4,000",22.90,23.20,-0.10,-0.43%,2,Buy,"91,600.00"
3,B,IMH,-2,RU09%,mai,"6,000",12.00,12.20,-0.40,-3.17%,2,Buy,"72,000.00"
15,B,WHART,-2,DOS,SET,"10,000",12.50,12.70,0.00,0.00%,2,Buy,"125,000.00"


In [17]:
show(df[['trans','name','spd','reason','qty','target','current','percent','active']])

trans,name,spd,reason,qty,target,current,percent,active


In [18]:
ms = (df.trans == 'Sell') & (df.spd <= 5)
df[ms].sort_values(['name'],ascending=[True]).style.format(format_dict)

,trade,name,spd,reason,market,qty,target,current,change,percent,active,trans,amount
17,S,DCC,2,3%,SET,"40,000",3.00,2.96,0.12,+4.23%,2,Sell,"120,000.00"
18,S,IVL,4,5%,SET50,"3,000",44.50,43.50,0.25,+0.58%,2,Sell,"133,500.00"


In [19]:
df[mb | ms].sort_values(['trade','name'],ascending=[True,True]).style.format(format_dict)

,trade,name,spd,reason,market,qty,target,current,change,percent,active,trans,amount
0,B,BGC,-3,DOS,SET,"10,000",10.40,10.70,0.10,+0.94%,2,Buy,"104,000.00"
1,B,DOHOME,-3,RD05%,SET100,"4,000",22.90,23.20,-0.10,-0.43%,2,Buy,"91,600.00"
3,B,IMH,-2,RU09%,mai,"6,000",12.00,12.20,-0.40,-3.17%,2,Buy,"72,000.00"
15,B,WHART,-2,DOS,SET,"10,000",12.50,12.70,0.00,0.00%,2,Buy,"125,000.00"
17,S,DCC,2,3%,SET,"40,000",3.00,2.96,0.12,+4.23%,2,Sell,"120,000.00"
18,S,IVL,4,5%,SET50,"3,000",44.50,43.50,0.25,+0.58%,2,Sell,"133,500.00"


In [20]:
df[mb | ms].shape[0]

6

In [21]:
mask = (df.trade == 'B')
df[mb | ms].nlargest(5, "amount")[["name","trans","amount"]].style.format(format_dict)

,name,trans,amount
18,IVL,Sell,"133,500.00"
15,WHART,Buy,"125,000.00"
17,DCC,Sell,"120,000.00"
0,BGC,Buy,"104,000.00"
1,DOHOME,Buy,"91,600.00"


In [22]:
df_out = df[mb | ms][colt]
#df_out = df[df.active == 2][colt]
df_out.style.format(format_dict)

,trans,name,qty,target,active,spd,current,change,percent
0,Buy,BGC,"10,000",10.40,2,-3,10.70,0.10,+0.94%
1,Buy,DOHOME,"4,000",22.90,2,-3,23.20,-0.10,-0.43%
3,Buy,IMH,"6,000",12.00,2,-2,12.20,-0.40,-3.17%
15,Buy,WHART,"10,000",12.50,2,-2,12.70,0.00,0.00%
17,Sell,DCC,"40,000",3.00,2,2,2.96,0.12,+4.23%
18,Sell,IVL,"3,000",44.50,2,4,43.50,0.25,+0.58%


In [23]:
file_name = 'ORDERS-PLACED.csv'
data_file = data_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name

df_out.sort_values(['trans','name'],ascending=[True,True]).to_csv(output_file, index=False)
df_out.sort_values(['trans','name'],ascending=[True,True]).to_csv(data_file, index=False)
df_out.sort_values(['trans','name'],ascending=[True,True]).to_csv(box_file, index=False)

### End of Output ORDERS-PLACED

In [24]:
sql = '''
SELECT trade, name, qty, price, qty * price AS amount, reason, market, active
FROM orders
WHERE ACTIVE = 2
ORDER BY trade, name'''
orders = pd.read_sql(sql, conlite)
orders.style.format(format_dict)

,trade,name,qty,price,amount,reason,market,active
0,B,BGC,"10,000",10.40,"104,000.00",DOS,SET,2
1,B,DOHOME,"4,000",22.90,"91,600.00",RD05%,SET100,2
2,B,IMH,"6,000",12.00,"72,000.00",RU09%,mai,2
3,B,ROJNA,"18,000",6.50,"117,000.00",6L,SET,2
4,B,WHART,"10,000",12.50,"125,000.00",DOS,SET,2
5,S,DCC,"40,000",3.00,"120,000.00",3%,SET,2
6,S,IVL,"3,000",44.50,"133,500.00",5%,SET50,2


### Evening Process

In [25]:
new_active = 1
old_active = 2

In [28]:
sql = """
UPDATE orders 
SET active = %s
WHERE active = %s
"""
sql = sql % (new_active, old_active)
rp = conlite.execute(sql)
rp.rowcount

7

In [29]:
names = df_out['name']
in_p = ", ".join(map(lambda name: "'%s'" % name, names))
in_p

"'BGC', 'DOHOME', 'IMH', 'WHART', 'DCC', 'IVL'"

In [30]:
sql = """
UPDATE orders 
SET active = 2
WHERE name IN (%s)
"""
sql = sql % (in_p)
rp = conlite.execute(sql)
rp.rowcount

6

### End of Evening Process